In [271]:
# _Definitions_

#!pip3 install pymerkle
#!pip install merlin-transcripts

from pymerkle import InmemoryTree as MerkleTree
from pymerkle import verify_inclusion
from merlin_transcripts import MerlinTranscript

N = 3 * 2^30 + 1
assert N.is_prime()

# Prime order field
F = GF(N)
# Plynomials field over F
R = F['x']

# Varriable to be used as a polynomial argument
x = F['x'].gen()

# F multiplicative group generator
w = F.primitive_element() # 5

In [272]:
def interpolate(poits):
    return R.lagrange_polynomial(points)

def build_tree(eval_domain):
    tree = MerkleTree(algorithm='sha256')
    for x in eval_domain:
        tree.append_entry(str(x).encode())
    return tree

def odd_f(poly):
    return R(R(poly).list()[1::2])
    
def even_f(poly):
    return R(R(poly).list()[::2])

def next_fri_domain(domain):
    return [F(i^2) for i in domain[:len(domain) // 2]]

def next_fri_polynomial(poly, beta):
    return even_f(poly) + beta * odd_f(poly)

def next_fri_layer(poly, domain, beta):
    next_f = next_fri_polynomial(poly, beta)
    next_domain = next_fri_domain(domain)
    return next_f, next_domain, [next_f(v) for v in next_domain]

In [273]:
# Get 1024 order multiplicative subgroup (domain to interpolate trace on)
g = F(w^(3 * 2^20))
G = [F(g^i) for i in range(1024)]

# Get 8192 order multiplicative subgroup (domain to evaluate `f` on)
h = F(w ^(3 * 2^17))
H = [F(h^i) for i in range(8192)]

# Build evaluation domain to evaluate polynomials on
eval_domain = [F(w * x) for x in H]

# _Finish definitions_

In [274]:
# Define the trace series to be proved
a = [F(1), F(3141592)] # I know a field element X such that the 1023rd element of the FibonacciSq sequence starting with 1, X is 2338775057.
while len(a) < 1023:
    a.append(a[-2] * a[-2] + a[-1] * a[-1])

In [275]:
# Build polynomial over trace `a`
points = [(G[i], a[i]) for i in range(1023)]
f = interpolate(points)

assert f(2) == F(1302089273), 'invalid interpolation'

In [276]:
# Evaluate `f` on our domain
f_eval = [f(d) for d in eval_domain]

In [277]:
# Create commitment for evaluated `f` using Merkle tree
f_tree = build_tree(f_eval)

In [278]:
# Encoding constraints

# First constraint a[0] = 1
p0 = (f - 1)/(x - 1)

# Second constraint a[1022] = 2338775057
p1 = (f - 2338775057) / (x - g^1022)

# Third constraint a[i] = a[i-1]^2 + a[i-2]^2
n2 = f(g^2 * x) - f(g * x)^2 - f^2
d2 = (x^1024 - 1) / ((x - g^1021) * (x - g^1022) * (x - g^1023))
p2 = n2/d2

In [279]:
prove_ts = MerlinTranscript(b"ZK-SNARK")
prove_ts.append_message(b"Trace commitment", f_tree.get_state())

a0 = F(int.from_bytes(prove_ts.challenge_bytes(b"a0", 32), byteorder='little'))
a1 = F(int.from_bytes(prove_ts.challenge_bytes(b"a1", 32), byteorder='little'))
a2 = F(int.from_bytes(prove_ts.challenge_bytes(b"a2", 32), byteorder='little'))

# Build Composition Polynomial
cp = a0*p0 + a1*p1 + a2*p2

# Evaluate Composition Polynomial on our domain
cp_eval = [cp(d) for d in eval_domain]

# Create commitment for evaluated `cp` using Merkle tree
cp_tree = build_tree(cp_eval)

prove_ts.append_message(b"CP commitment", cp_tree.get_state())

fri_poly = cp
fri_domain = eval_domain
fri_merkles = [cp_tree]
fri_evals = [cp_eval]

# Execute FRI protocol
while F['x'](fri_poly).degree() > 0:
    beta = F(int.from_bytes(prove_ts.challenge_bytes(b"beta", 32), byteorder='little'))
    next_poly, next_domain, next_layer = next_fri_layer(fri_poly, fri_domain, beta)    
    next_layer_tree = build_tree(next_layer)
    fri_merkles.append(next_layer_tree)
    fri_evals.append(next_layer)
    fri_poly = next_poly
    fri_domain = next_domain
    prove_ts.append_message(b"FRI commitment", next_layer_tree.get_state())

# Get challange index
t = int.from_bytes(prove_ts.challenge_bytes(b"challenge", 32), byteorder='little') % len(H)

assert t + 16 < len(H)

f_x_proof = f_tree.prove_inclusion(t+1)
f_gx_proof = f_tree.prove_inclusion(t+9)
f_ggx_proof = f_tree.prove_inclusion(t+17)

proof_fri_merkle_path = [[cp_tree.prove_inclusion(t+1), cp_tree.prove_inclusion((t + len(H)/2) % len(H) + 1)]]
proof_cp = [[cp_eval[t], cp_eval[(t + len(H)/2) % len(H)]]]

idx = t
layer_len = len(H)
for i in range(1, len(fri_merkles)):    
    layer_len = layer_len/2
    idx = idx % layer_len
    midx = (idx + layer_len/2) % layer_len
   
    new_cx_proof = fri_merkles[i].prove_inclusion(idx+1)
    new_cmx_proof = fri_merkles[i].prove_inclusion(midx + 1) 
    
    proof_fri_merkle_path.append([new_cx_proof, new_cmx_proof])
    proof_cp.append([fri_evals[i][idx], fri_evals[i][midx]])


# Proof
proof_fx = f_eval[t]
proof_fgx = f_eval[t+8] # g(wh^t) = w * w^(3*2^20) * w^(t*3*2^17) = w * w^3*2^17(t + 8) = wh^(t+8)
proof_fggx = f_eval[t+16]
proof_f_com = f_tree.get_state()
proof_fx_path = f_tree.prove_inclusion(t+1)
proof_fgx_path = f_tree.prove_inclusion(t+9)
proof_fggx_path = f_tree.prove_inclusion(t+17)
proof_fri_merkle_path
proof_cp
proof_fri_com = [tree.get_state() for tree in fri_merkles]

In [280]:
# _Verification_

assert len(proof_cp) < 12

# Check all Merkle pathes
f_tree = build_tree([proof_fx])
verify_inclusion(f_tree.get_leaf(1), proof_f_com, proof_fx_path)

f_tree = build_tree([proof_fgx])
verify_inclusion(f_tree.get_leaf(1), proof_f_com, proof_fgx_path)

f_tree = build_tree([proof_fggx])
verify_inclusion(f_tree.get_leaf(1), proof_f_com, proof_fggx_path)

for i in range(len(proof_cp)):
    cp_tree = build_tree([proof_cp[i][0]])
    verify_inclusion(cp_tree.get_leaf(1), proof_fri_com[i], proof_fri_merkle_path[i][0])

    cp_tree = build_tree([proof_cp[i][1]])
    verify_inclusion(cp_tree.get_leaf(1), proof_fri_com[i], proof_fri_merkle_path[i][1])


verify_ts = MerlinTranscript(b"ZK-SNARK")

verify_ts.append_message(b"Trace commitment", proof_f_com)

a0 = F(int.from_bytes(verify_ts.challenge_bytes(b"a0", 32), byteorder='little'))
a1 = F(int.from_bytes(verify_ts.challenge_bytes(b"a1", 32), byteorder='little'))
a2 = F(int.from_bytes(verify_ts.challenge_bytes(b"a2", 32), byteorder='little'))

verify_ts.append_message(b"CP commitment", proof_fri_com[0])

beta = []

for com in proof_fri_com[1:]:
    b = F(int.from_bytes(verify_ts.challenge_bytes(b"beta", 32), byteorder='little'))
    beta.append(b)
    verify_ts.append_message(b"FRI commitment", com)

t = int.from_bytes(verify_ts.challenge_bytes(b"challenge", 32), byteorder='little') % len(H)

# Build `cp(x)` and verify user calculation
p0_x = F((proof_fx - 1)/(w*h^t - 1))
p1_x = F((proof_fx - 2338775057) / (w*h^t - g^1022))

n2 = F(proof_fggx - proof_fgx^2 - proof_fx^2)
d2 = ((w*h^t)^1024 - 1) / ((w*h^t - g^1021) * (w*h^t - g^1022) * (w*h^t - g^1023))
p2_x = F(n2/d2)

cp_x = a0*p0_x + a1*p1_x + a2*p2_x
assert cp_x == proof_cp[0][0]

# Verify FRI
cp_mx = proof_cp[0][1]
cp_domain = eval_domain
idx = t

for i in range(1, len(proof_cp)):
    gx = F((proof_cp[i-1][0]+proof_cp[i-1][1])/2)
    hx = F((proof_cp[i-1][0]-proof_cp[i-1][1])/(2*cp_domain[idx]))
    new_cp_x = F(gx + beta[i-1]*hx)
    assert new_cp_x == proof_cp[i][0]
    cp_domain = next_fri_domain(cp_domain)
    idx = idx % len(cp_domain)